In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd     
import gsw
import os

In [4]:
path = r'G:\最新数据\截面\N21'
wen_jian_jia = os.listdir(path)
print(wen_jian_jia)

['CTD-N21-14.xls', 'CTD-N21-10.xls', 'CTD-N21-11.xls', 'CTD-N21-12.xls', 'CTD-N21-13.xls']


In [5]:
def dms_to_decimal(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [8]:
plt.rcParams['font.size'] = 18
all_colors = ['#8B8970', '#FFA54F', '#3CB371', '#8B2500', '#B0C4DE', '#8B4513', '#0FFFF0', '#6B8E23', '#F08080', '#008080', '#BC8F8F', '#FFC0CB', '#9932CC', '#F0E68C', '#B22222']
all_markers = ['o','o','o','D','D','D','X','X','X','*','*','*','^','^','^']

fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(111)

for idx, path_name in enumerate(wen_jian_jia):
    colors = all_colors[:len(wen_jian_jia)]
    df = pd.read_excel(rf'G:\最新数据\截面\N21\{path_name}', skiprows=3, usecols=[1,5,6,8,10,11,13,16], names=['站位名', '纬度', '经度', '深度', '压力', '水温', '盐度', '溶解氧'])
    df = df[df['深度'] >= 2000]
    o2 = df['溶解氧'].values.astype(float)
    salinity = df['盐度'].values.astype(float)
    t = df['水温'].values.astype(float)
    p = df['压力'].values.astype(float) * 0.1 -10.1325
    lon = df['经度'].apply(dms_to_decimal).values.astype(float)
    lat = df['纬度'].apply(dms_to_decimal).values.astype(float)
    
    AT = gsw.SA_from_SP(salinity, p, lon, lat)
    PT = gsw.pt0_from_t(AT, t, p)
    CT = gsw.CT_from_pt(AT, PT)
    sigma = gsw.sigma0(AT, CT)
    
    ax1.scatter(salinity, PT, s=20, marker=all_markers[idx], c=colors[idx], alpha=0.5, label=path_name[4:10])

plt.legend(markerscale=2, fontsize=15)
# plt.show()
plt.savefig('N21', bbox_inches='tight', dpi=300)
plt.close()    